In [7]:
import gradio as gr
import requests

def chat_with_model(message, history):
    """
    Send message to local Llama model and return response
    """
    # Ollama API endpoint for generate
    url = "http://localhost:11434/api/generate"
    
    # Build conversation context
    conversation = ""
    for user_msg, bot_msg in history:
        conversation += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    conversation += f"User: {message}\nAssistant:"
    
    # Prepare request payload for Ollama
    payload = {
        "model": "llama3.2:latest",
        "prompt": conversation,
        "stream": False
    }
    
    try:
        # Send request to Ollama
        response = requests.post(url, json=payload, timeout=120)
        
        # Check for errors and get detailed info
        if response.status_code != 200:
            try:
                error_detail = response.json()
                return f"❌ Error {response.status_code}: {error_detail}"
            except:
                return f"❌ Error {response.status_code}: {response.text}"
        
        # Parse response
        result = response.json()
        bot_response = result.get("response", "No response received")
        
        return bot_response
    
    except requests.exceptions.ConnectionError:
        return "❌ Error: Cannot connect to Ollama. Make sure Ollama is running with 'ollama serve'"
    except requests.exceptions.Timeout:
        return "❌ Error: Request timed out. The model might be taking too long to respond."
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Create Gradio ChatInterface
demo = gr.ChatInterface(
    fn=chat_with_model,
    title="🤖 Llama 3.2 Chatbot",
    description="Chat with your local Llama 3.2 model (2.0 GB)",
    examples=[
        "Hello! How are you?",
        "Explain quantum computing in simple terms",
        "Write a short poem about AI"
    ]
)

# Launch the interface
if __name__ == "__main__":
    demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
